# Identity Access Management
API documentation:
- [IAM identity services](https://cloud.ibm.com/apidocs/iam-identity-token-api)<br/>
This documentation includes API description for Java, Node, Python, and Go interfaces

The functionality demonstrated includes:
- Defining Python function for REST API calls
- Bearer token generation and header file definition
- Get the details of the API key using the REST API
- Get some key details using the Python interface
- Get a specific API key details using its id through the Python interface
- Get the account configuration using the REST API
- Get the account configuration using the Python interface

## Preparation
Don't forget to generate an API key and update the `API_key` variable in the cell below.

In some cases, we are using both the REST API and the equivalent Python library.

In [ ]:
import requests
import json
import warnings
import os
from datetime import datetime

# from ibm_watson_studio_lib import access_project_or_space

IAM_ENDPOINT = "https://iam.cloud.ibm.com"
API_key = "<Insert API key here>" # from cloud.ibm.com...

# Should not use verify=False but I don't want to deal with SSL
verify=False
warnings.filterwarnings("ignore") # one of "error", "ignore", "always", "default", "module", or "once"

In [ ]:
# IAM identity services Python API interface
!pip install --upgrade "ibm-platform-services" > /dev/null 2>&1
os.environ["IAM_IDENTITY_URL"] = IAM_ENDPOINT
os.environ["IAM_IDENTITY_AUTHTYPE"] = "iam"
os.environ["IAM_IDENTITY_APIKEY"] = API_key

from ibm_platform_services import IamIdentityV1

service_client = IamIdentityV1.new_instance()
print("Service name: {}".format(service_client.DEFAULT_SERVICE_NAME))

## Support functions

In [ ]:
def getToken(key) :
    """Get the access token required to interface with CPDaaS"""
    headers = {
        'Accept': 'application/json',
        'Content-type': 'application/x-www-form-urlencoded'
    }
    data = "grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={}"
    resp = requests.post(IAM_ENDPOINT + '/identity/token', 
                        headers=headers, data=data.format(key))
    return(resp)

def apikeyDetails(API_key, access_token) :
    headers = {
        'Authorization': 'Bearer ' + access_token,
        'IAM-Apikey' : API_key
    }
    resp = requests.get(IAM_ENDPOINT + '/v1/apikeys/details', headers=headers)
    return(resp)

def accountConfig(headersAPI, account_id) :
    endpoint = IAM_ENDPOINT + '/v1/accounts/{}/settings/identity?include_history=false'
    resp = requests.get(endpoint.format(account_id), headers=headersAPI)
    return(resp)


## Generate the access token/bearer key
The token needs to be regenerated **every hour**

In [ ]:
resp = getToken(API_key)
print("Status code: {}, reason: {}".format(resp.status_code,resp.reason))
print("Got a token at {} GMT".format(datetime.now().time().isoformat("seconds")))

access_token = resp.json()['access_token']

headersAPI = {
    'accept': 'application/json',
    'Content-type': 'application/json',
    'Authorization': 'Bearer ' + access_token,
    'cache-control': 'no-cache'
}

### Other ways to generate an access token
- Using username/password: only works for non-federated IBMid users (what does that mean?)
- Using an authorization policy. Typically used for access between services
- Create a access token and a delegated refresh token for an API key
- Create a non-opaque access token without a refresh token for a Trusted Profile

## Get the details of the API key
This will give us additional information such as:
- account ID
- Identity of the user (iam_id)

These values are needed for some of the calls.

In [ ]:
resp = apikeyDetails(API_key, access_token)
print("Status code: {}, reason: {}".format(resp.status_code,resp.reason))
key_details_json = resp.json()

account_id = key_details_json['account_id']
iam_id = key_details_json['iam_id']

print(json.dumps(key_details_json, indent=2, sort_keys=False))

## Retrieve some API key details using the Python library
This is not equivalent to the previous call.
- `list_api_keys`: Get a list of the API keys for this account. The default returns up to 20 keys.<br/>
The maximum number of keys returned is 100. To get more results, additional calls have to be made.
- `get_api_key`: retrieve the details of a specific API key using its `id` 
which is different from the key itself

In [ ]:
# Retrieve the first 20 keys and display the first one
resp = service_client.list_api_keys(pagesize=20, type="user")
resp_json = resp.get_result()
# Display one key
print(json.dumps(resp_json['apikeys'][0], indent=2, sort_keys=False))

In [ ]:
# Retrieve the API key details using the id returned through the REST API earlier
resp = service_client.get_api_key(id=key_details_json['id'])
resp.get_result()

## Get the account configuration

In [ ]:
resp = accountConfig(headersAPI, account_id)
print("Status code: {}, reason: {}".format(resp.status_code,resp.reason))
print(json.dumps(resp.json(), indent=2, sort_keys=False))

In [ ]:
# Getting the account configuration using the Python library
resp = service_client.get_account_settings(account_id=account_id)
print(json.dumps(resp.result, indent=2, sort_keys=False))

### Author
**Jacques Roy** is a member of the IBM Enablement for Data and AI

Copyright © 2023. This notebook and its source code are released under the terms of the MIT License.